In [1]:
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
import sklearn
import os, pathlib, shutil
import kaggle, kagglehub


Download the competition


In [2]:
competition_name = "home-data-for-ml-course"  # Change this to the competition you want to download
competition_path = pathlib.Path(
    kagglehub.competition.competition_download(competition_name)
)  # Downloads the competition
competition_files: list = os.listdir(competition_path)  # List of the files included in the competition

print(competition_files)

# copy files to the current directory
shutil.copyfile(competition_path / "sample_submission.csv", "sample_submission.csv")
shutil.copyfile(competition_path / "test.csv", "test.csv")
shutil.copyfile(competition_path / "train.csv", "train.csv")

['data_description.txt', 'sample_submission.csv', 'sample_submission.csv.gz', 'test.csv', 'test.csv.gz', 'train.csv', 'train.csv.gz']


'train.csv'

In [3]:
pl.Config.set_tbl_rows(50)
train_df = pl.read_csv(competition_path / "train.csv", infer_schema_length=None)
test_df = pl.read_csv(competition_path / "test.csv", infer_schema_length=None)
sample_submission_df = pl.read_csv(competition_path / "sample_submission.csv")

target_feature_name: str = (set(train_df.columns) - set(test_df.columns)).pop()
target_col = train_df.select(pl.col(target_feature_name))
# print(train_df.schema == test_df.schema)
# df = pl.concat([train_df, test_df])

### EDA


In [55]:
numeric_cols_df = train_df.drop("Id").select(pl.selectors.numeric())
numeric_cols_df.select(pl.corr(pl.col("*"), target_feature_name).abs()).transpose(include_header=True).sort(
    "column_0", descending=True
)

column,column_0
str,f64
"""SalePrice""",1.0
"""OverallQual""",0.790982
"""GrLivArea""",0.708624
"""GarageCars""",0.640409
"""GarageArea""",0.623431
…,…
"""YrSold""",0.028923
"""LowQualFinSF""",0.025606
"""MiscVal""",0.02119
